### Control Digital en Sistemas Embebidos - MSE - PRACTICA 1

In [ ]:
!pipenv install slycot
!pipenv install control
!pipenv install matplotlib

In [ ]:
import numpy as np
from scipy.linalg import expm
from control import *
import matplotlib.pyplot as plt

In [ ]:
r_1 = 10e3
c_1 = 10e-6
r_2 = 1e3
c_2 = 10e-6

**1)** Crear una planta mediante los coeficientes del denominador y numerador en el dominio continuo.

**Opcion 1:** Realizar el desarrollo matematico de la ecuación diferencial del circuito y obtener la función transferencia en el dominio de Laplace

In [ ]:
s = tf('s')
hs_1 = 1 / ((r_1*c_1*r_2*c_2)*s*s + (r_1*c_1+r_1*c_2+r_2*c_2)*s + 1)
num_1, den_1 = tfdata(hs_1)
num_1, den_1


**Opcion 2:** Realizar el desarrollo matematico de la ecuación diferencial del circuito para obtener los coeficientes del numerador y denominador

In [ ]:
num_2 = np.array([1])
den_2 = np.array([(r_1*c_1*r_2*c_2), (r_1*c_1+r_1*c_2+r_2*c_2), 1])
hs_2 = tf(num_2, den_2)
hs_2

**2)** Crear una planta mediante las matrices de estado (A, B, C y D) que la definen.

Se pueden obtener los coeficientes de las ecuaciones de estado a partir de los coeficientes del numerador y denominador, y la funcion 'tf2ss'.

In [ ]:
ss_1 = tf2ss(num_1, den_1)
ss_1

Se pueden obtener los coeficientes del numerador y denominador de la funcion transferencia a partir de los coeficientes de estado con 'ss2tf'

In [ ]:
hs_2 = ss2tf(ss_1)
hs_2

In [ ]:
hs_3 = ss2tf(ss_1.A, ss_1.B, ss_1.C, ss_1.D)
hs_3

También puede obtenerse la función transferencia a partir de los coeficientes de las ecuaciones de estado con: 

$H_{s} = C (sI − A)^{−1} B + D$

In [ ]:
s = tf('s')
hs_3 = ss_1.C / (np.multiply(np.eye(ss_1.A.shape[0]), s) - ss_1.A) * ss_1.B + ss_1.D
hs_3

**3)** Discretizar las anteriores plantas

In [ ]:
h = 0.1

**Opcion 1:** Discretizar utilizando la funcion 'c2d'

In [ ]:
hz_1 = c2d(hs_1, h, 'zoh')
numz_1, denz_1 = tfdata(hz_1)
ss_2 = tf2ss(numz_1, denz_1)
ss_2

**Opcion 2:** Discretizar utilizando las funciones para obtener las ecuaciones de estado discretas a partir de los coeficientes de las ecuaciones en tiempo continuo.

In [ ]:
phi_2 = expm(ss_1.A * h)
rho_2 = (expm(ss_1.A * h) - expm(ss_1.A * 0)) / ss_1.A * ss_1.B
cz_2 = ss_1.C
dz_2 = 0

ss2tf(phi_2, rho_2, cz_2, dz_2)
rho_2

**4)** Evaluar la distribución de polos y ceros de las plantas continuas y discretas, también su respuesta en frecuencia.

In [ ]:
bode_plot(hs_1);

In [ ]:
zeros(hs_1), poles(hs_1)

In [ ]:
zeros(hz_1), poles(hz_1)

In [ ]:
pzmap(hz_1)

**5)** Evaluar la respuesta al escalón de todas las plantas (continuas y discretas)

In [ ]:
t, hs_out = step_response(hs_1)
plt.plot(t, hs_out);

In [ ]:
t, hz_out = step_response(hz_1)
plt.plot(t, hz_out);

**6)** Crear una función que aplique la ecuación de recurrencia de un filtro a una entrada cualquiera. (Ver función filter de Matlab).

In [ ]:
def funcion_recurrencia(num: np.ndarray, den: np.ndarray, input: np.ndarray) -> np.ndarray:
    # TODO
    return input

**7)** Compare las respuestas al escalón discretas con el comando step frente a la creada con la ecuación de recurrencia

In [ ]:
t = np.linspace(h, h * 100, 100)
u = np.concatenate((np.zeros(int(len(t) / 2)), np.ones(int(len(t) / 2))), axis=None)

In [ ]:
y_1 = funcion_recurrencia(numz_1, denz_1, u)
t_2, y_2 = input_output_response(tf2io(hz_1), t, u)

In [ ]:
p_1 = plt.plot(y_1)
p_2 = plt.plot(y_2)
plt.ylabel('u(t)')
plt.title('Respuesta al Escalon')
plt.legend((p_1[0], p_2[0]), ('y1(t)', 'y2(t)'));